## THIS IS THE HOLY GRAIL THAT IS GONNA SAVE OUR ASSES 
(https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)

## Constructing a shallow ANN for classification
##MAYBE add in Sentiment Analysis as feature 

In [0]:
!pip3 install pandas keras numpy xgboost sklearn nltk textblob

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
cd drive/'My Drive'/CS229

### Get genuine human tweets

In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, xgboost, numpy as np, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

real_users = pd.read_csv('./GenuineData/ga/users.csv',low_memory = False)
real_tweets = pd.read_csv('./GenuineData/ga/tweets.csv',low_memory = False)
real_df = real_tweets.merge(real_users, on='id',how = 'outer', suffixes = ('_tweets','_users')) #Merging the user and tweets
real_df['labels']=pd.Series(np.ones(len(real_df['id']))) #Creating a new column to indicate that this is human data


In [0]:
real_df['timestamp_tweets']

In [0]:
import tensorflow as tf
tf.test.gpu_device_name() #We good using GPU

### Import Bot Dataset from this point on 
(https://github.com/fivethirtyeight/russian-troll-tweets/)

In [0]:
import os
directory = os.fsencode('./russian-troll-tweets')
troll_data = pd.DataFrame()
for file in os.listdir(directory):
  print(file)
  filename = os.fsdecode(file)
  if filename.endswith(".csv"):
    botdata = pd.read_csv('./russian-troll-tweets/'+filename,low_memory = False)
    troll_data = troll_data.append(botdata) #append together

In [0]:
troll_data = troll_data.rename(index = str, columns = {"publish_date":"date"})
#troll_data[troll_data['publish_date'].year >=2017]


In [0]:
frac = 80       #Factor to Downsample due to runtime constraints 
troll_data['labels']=pd.Series(np.zeros(len(botdata)))
#Limiting too only English language tweets
english_bot = troll_data[troll_data['language']  == 'English']
troll_data['language'].unique()
print(len(english_bot),len(troll_data)) #Class imbalance though, we need more spam bot results
train_df = pd.DataFrame() #Create train data frame for splitting later
real_df = real_df.rename(index = str, columns = {"timestamp_tweets": "date"})
train_df = train_df.append(real_df[['text','labels','date']].sample(int(len(real_df)/frac)))
english_bot = english_bot.rename(index=str, columns={"content": "text"})
train_df = train_df.append(english_bot[['text','labels','date']].sample(int(len(english_bot)/frac)))
#Downsample to 12.5%
len(train_df)

In [0]:
train_df.head()

### Split train test dataset:

In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_df['text'], train_df['labels'], test_size = .2)

### Create Count Vectors for NB

In [0]:
#fill in nan to make sklearn happy
train_df['text'] = train_df['text'].fillna(' ')
train_x = train_x.fillna(' ')
valid_x = valid_x.fillna(' ') 
#ngram
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_df['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)


In [0]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


In [0]:
#xtrain_count.shape


### Shallow Neural Net that uses count vector and tf-IDF and possible sentiment

### Train Model of select type and make predictions. 

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    classifier.fit(feature_vector_train, label)
    pred_train = classifier.predict(feature_vector_train)
    predictions = classifier.predict(feature_vector_valid)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
        pred_train = pred_train.argmax(axis = -1)

    return metrics.classification_report(label, pred_train), metrics.classification_report(valid_y, predictions)#confusion matrix on validation matrix
  #metrics.confusion_matrix(label, pred_test), metrics.confusion_matrix(valid_y, predictions)

In [0]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer1 = layers.Dense(100, activation='relu')(input_layer)
    hidden_layer2 = layers.Dense(20, activation="relu")(hidden_layer1)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer2)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
train,test = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)


In [0]:
#USE THIS INSTEAD
#https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
sklearn.neural_network.MLPClassifier

In [0]:
print(test)

### Naive Bayes

## Build 2 layer Neural Net, tweek later

In [0]:
# Naive Bayes on Count Vectors
np.nan_to_num(xtrain_count)
np.nan_to_num(xvalid_count)
conf_test, conf_valid = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print (conf_test[0:2])
print(conf_valid[0:2])

'''# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print "NB, WordLevel TF-IDF: ", accuracy

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "NB, N-Gram Vectors: ", accuracy

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
'''

In [0]:
print (conf_test[:2])
print(conf_valid[:2])

tf-idf: don't run yet

In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 